In [11]:
import keras
import numpy as np
import random

In [14]:
data=np.load("deepwriting_training.npz")
texxt=data['texts']
text = ''.join(texxt)
print(len(text))

1040258


In [16]:
text

'He smiled at his ownactually dripping from his hands .problems. They will not work inconservatives have been contentEach section of thewhile occupying long chairs side byBowman could only rampage and roisterthat he had been swirled down a bumpy lift.Peaslake planted his head ingrape-vines, blackberries and other fruitsindescribably happy face. After acontained one or more dwellings .dishes can be transferred"You begin newly here. Where are your souls?"Acknowledgments. A graphicalhad no right to dictate to her,The colonial Secretary, Mr. Lennoxfor me, and tell her I \'m sorry Iseat opposite without making anyfurniture, fabrics and carpets.podzolised soils associated withFor Sam ought to be prejudiced. Sam wassounds very kind of him. "met your neighbour yet?"in the hall holding a tray for cards -relationship began in the primary School.the booking counting allotteda moment in disgust. "And that \'sMrs. Lawford. There are aappetite and added a more exquisite and etherealhad gone bunkrapt

In [17]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 81


In [18]:
sentences = []
next_chars = []
for i in range(0, len(text) - 40, 3):
    sentences.append(text[i: i + 40])
    next_chars.append(text[i + 40])

x = np.zeros((len(sentences), 40, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [19]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(40, len(chars))))
model.add(keras.layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               107520    
                                                                 
 dense_1 (Dense)             (None, 81)                10449     
                                                                 
Total params: 117969 (460.82 KB)
Trainable params: 117969 (460.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [22]:
model.fit(x, y, batch_size=18, epochs=1)
start_index = random.randint(0, len(text) - 40 - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    generated = ''
    sentence = text[start_index: start_index + 40]
    generated += sentence

    for i in range(400):
        x_pred = np.zeros((1, 40, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
with open('example.txt', 'w') as f:
    f.write(generated)

19264/19264 [==============================] - 329s 17ms/step - loss: 2.2355


In [23]:
from google.colab import files
files.download('example.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>